## Aggregate Function Example
> Time-series aggregation function   
  performs a rolling aggregation on {df_artifact}, over {window} by the selected {keys}
  applying {metric_aggs} on {metrics} and {label_aggs} on {labels}. adding {suffix} to the 
  feature names.
    
    

In [1]:
from mlrun import import_function

In [2]:
# declare the dataset
DATA = "https://s3.wasabisys.com/iguazio/data/market-palce/aggregate/metrics.pq"

In [3]:
# import the function
aggregate_function = import_function("hub://aggregate")

In [4]:
aggregate_function.run(params={'metrics': ['cpu_utilization'],
               'labels': ['is_error'],
               'metric_aggs': ['mean', 'sum'],
               'label_aggs': ['max'],
               'suffix': 'daily',
               'inplace': False,
               'window': 5,
               'center': True,               
               'files_to_select': 2}           
           , inputs={'df_artifact': DATA}
           )
    

> 2021-07-25 12:01:50,430 [info] starting run aggregate-aggregate uid=3bca51c563ad40f786b83d20205709b3 DB=http://mlrun-api:8080
> 2021-07-25 12:01:50,510 [info] Job is running in the background, pod: aggregate-aggregate-rmp56
> 2021-07-25 12:01:53,741 [info] Aggregating https://s3.wasabisys.com/iguazio/data/market-palce/aggregate/metrics.pq
> 2021-07-25 12:01:55,024 [error] Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/mlrun/runtimes/local.py", line 351, in exec_from_params
    val = handler(*args_list)
  File "main.py", line 68, in aggregate
    input_df = df_artifact.as_df()
  File "/usr/local/lib/python3.7/site-packages/mlrun/datastore/base.py", line 342, in as_df
    **kwargs,
  File "/usr/local/lib/python3.7/site-packages/mlrun/datastore/base.py", line 193, in as_df
    if fs.isdir(url):
  File "/usr/local/lib/python3.7/site-packages/fsspec/asyn.py", line 72, in wrapper
    return sync(self.loop, func, *args, **kwargs)
  File "/usr/local/lib/pyt

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...205709b3,0,Jul 25 12:01:53,error,aggregate-aggregate,kind=jobowner=jovyanhost=aggregate-aggregate-rmp56,df_artifact,"metrics=['cpu_utilization']labels=['is_error']metric_aggs=['mean', 'sum']label_aggs=['max']suffix=dailyinplace=Falsewindow=5center=Truefiles_to_select=2",,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 3bca51c563ad40f786b83d20205709b3 --project default , !mlrun logs 3bca51c563ad40f786b83d20205709b3 --project default
> 2021-07-25 12:01:56,623 [info] run executed, status=error
runtime error: 'utf-8' codec can't decode byte 0x80 in position 7: invalid start byte


RunError: 'utf-8' codec can't decode byte 0x80 in position 7: invalid start byte

### Test on cluster

Convert the code to an MLRun function

In [ ]:
aggregate_run = fn.apply(mount_v3io()).run(aggregate_task, artifact_path=os.path.abspath('./'))

### Show results

In [91]:
pd.read_parquet(aggregate_run.artifact('aggregate')['target_path'])

cpu_utilization  \
timestamp               company                     data_center      device                           
2020-06-30 06:42:12.454 Wilson_LLC                  Obrien_Mountain  0966571261270        80.900460   
                                                                     8069812479542        82.345824   
                        Bennett__Delacruz_and_Walls Natasha_Harbors  5863502247054        85.551250   
                                                                     4285071567351        68.295606   
                                                    Dominique_Branch 4579248894449        78.399353   
...                                                                                             ...   
2020-06-30 07:42:12.454 Wilson_LLC                  Zachary_Drives   6001003522699       100.000000   
                                                    Obrien_Mountain  0966571261270        62.157867   
                                                                     8069812479542        59.574487   
                        Bennett__Delacruz_and_Walls Natasha_Harbors  5863502247054       100.000000   
                                                                     4285071567351       100.000000   

                                                                                       latency  \
timestamp               company                     data_center      device                      
2020-06-30 06:42:12.454 Wilson_LLC                  Obrien_Mountain  0966571261270    3.696670   
                                                                     8069812479542    7.308960   
                        Bennett__Delacruz_and_Walls Natasha_Harbors  5863502247054    0.000000   
                                                                     4285071567351    7.528826   
                                                    Dominique_Branch 4579248894449    0.000000   
...                                                                                        ...   
2020-06-30 07:42:12.454 Wilson_LLC                  Zachary_Drives   6001003522699  100.000000   
                                                    Obrien_Mountain  0966571261270    1.911734   
                                                                     8069812479542    0.000000   
                        Bennett__Delacruz_and_Walls Natasha_Harbors  5863502247054  100.000000   
                                                                     4285071567351  100.000000   

                                                                                    packet_loss  \
timestamp               company                     data_center      device                       
2020-06-30 06:42:12.454 Wilson_LLC                  Obrien_Mountain  0966571261270     0.000000   
                                                                     8069812479542     2.567228   
                        Bennett__Delacruz_and_Walls Natasha_Harbors  5863502247054     0.000000   
                                                                     4285071567351     0.000000   
                                                    Dominique_Branch 4579248894449     0.000000   
...                                                                                         ...   
2020-06-30 07:42:12.454 Wilson_LLC                  Zachary_Drives   6001003522699    50.000000   
                                                    Obrien_Mountain  0966571261270     1.253481   
                                                                     8069812479542     0.000000   
                        Bennett__Delacruz_and_Walls Natasha_Harbors  5863502247054    50.000000   
                                                                     4285071567351    50.000000   

                                                                                    throughput  \
timestamp               company                     data_center      device        